# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [1]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [2]:
data_file="reviews_data.txt.gz"

with gzip.open ('reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [3]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2018-10-23 14:20:45,670 : INFO : reading file reviews_data.txt.gz...this may take a while
2018-10-23 14:20:45,672 : INFO : read 0 reviews
2018-10-23 14:20:47,824 : INFO : read 10000 reviews
2018-10-23 14:20:49,912 : INFO : read 20000 reviews
2018-10-23 14:20:52,373 : INFO : read 30000 reviews
2018-10-23 14:20:54,715 : INFO : read 40000 reviews
2018-10-23 14:20:57,520 : INFO : read 50000 reviews
2018-10-23 14:21:00,099 : INFO : read 60000 reviews
2018-10-23 14:21:03,006 : INFO : read 70000 reviews
2018-10-23 14:21:05,371 : INFO : read 80000 reviews
2018-10-23 14:21:07,852 : INFO : read 90000 reviews
2018-10-23 14:21:10,561 : INFO : read 100000 reviews
2018-10-23 14:21:12,706 : INFO : read 110000 reviews
2018-10-23 14:21:14,900 : INFO : read 120000 reviews
2018-10-23 14:21:17,227 : INFO : read 130000 reviews
2018-10-23 14:21:19,796 : INFO : read 140000 reviews
2018-10-23 14:21:22,375 : INFO : read 150000 reviews
2018-10-23 14:21:25,925 : INFO : read 160000 reviews
2018-10-23 14:21:28,386

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [4]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10, sg=0)
model.train(documents,total_examples=len(documents),epochs=10)

2018-10-23 14:26:31,264 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-10-23 14:26:31,266 : INFO : collecting all words and their counts
2018-10-23 14:26:31,267 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-10-23 14:26:31,548 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2018-10-23 14:26:31,809 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2018-10-23 14:26:32,200 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2018-10-23 14:26:32,515 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2018-10-23 14:26:32,873 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2018-10-23 14:26:33,200 : INFO : PROGRESS: at sentence #60000, processed 11013723 words, keeping 76781 word types
2018-10-23 14:26:33,462 : INFO : PROGRESS: a

2018-10-23 14:27:11,747 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-10-23 14:27:11,749 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-10-23 14:27:11,757 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-10-23 14:27:11,762 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-10-23 14:27:11,762 : INFO : EPOCH - 1 : training on 41519355 raw words (30350189 effective words) took 31.7s, 957056 effective words/s
2018-10-23 14:27:12,784 : INFO : EPOCH 2 - PROGRESS: at 2.92% examples, 901437 words/s, in_qsize 20, out_qsize 0
2018-10-23 14:27:13,833 : INFO : EPOCH 2 - PROGRESS: at 5.99% examples, 900507 words/s, in_qsize 20, out_qsize 0
2018-10-23 14:27:14,834 : INFO : EPOCH 2 - PROGRESS: at 9.06% examples, 922998 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:27:15,838 : INFO : EPOCH 2 - PROGRESS: at 10.91% examples, 864773 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:27:16,843 : INFO : EPOCH 2 - 

2018-10-23 14:28:13,396 : INFO : EPOCH 3 - PROGRESS: at 86.02% examples, 867927 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:28:14,400 : INFO : EPOCH 3 - PROGRESS: at 89.18% examples, 868009 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:28:15,405 : INFO : EPOCH 3 - PROGRESS: at 92.62% examples, 871385 words/s, in_qsize 18, out_qsize 1
2018-10-23 14:28:16,429 : INFO : EPOCH 3 - PROGRESS: at 95.93% examples, 873856 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:28:17,442 : INFO : EPOCH 3 - PROGRESS: at 98.89% examples, 872823 words/s, in_qsize 18, out_qsize 1
2018-10-23 14:28:17,689 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-10-23 14:28:17,690 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-10-23 14:28:17,690 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-10-23 14:28:17,693 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-10-23 14:28:17,715 : INFO : worker thread finished; awaiting 

2018-10-23 14:29:09,188 : INFO : EPOCH 5 - PROGRESS: at 31.62% examples, 792473 words/s, in_qsize 20, out_qsize 0
2018-10-23 14:29:10,192 : INFO : EPOCH 5 - PROGRESS: at 34.68% examples, 800070 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:29:11,213 : INFO : EPOCH 5 - PROGRESS: at 37.40% examples, 797514 words/s, in_qsize 17, out_qsize 2
2018-10-23 14:29:12,237 : INFO : EPOCH 5 - PROGRESS: at 40.18% examples, 794693 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:29:13,255 : INFO : EPOCH 5 - PROGRESS: at 41.96% examples, 774892 words/s, in_qsize 20, out_qsize 1
2018-10-23 14:29:14,271 : INFO : EPOCH 5 - PROGRESS: at 44.32% examples, 768103 words/s, in_qsize 16, out_qsize 3
2018-10-23 14:29:15,283 : INFO : EPOCH 5 - PROGRESS: at 47.49% examples, 774839 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:29:16,284 : INFO : EPOCH 5 - PROGRESS: at 50.74% examples, 782465 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:29:17,287 : INFO : EPOCH 5 - PROGRESS: at 53.19% examples, 781373 words/s,

2018-10-23 14:30:10,190 : INFO : EPOCH 1 - PROGRESS: at 87.91% examples, 779062 words/s, in_qsize 18, out_qsize 1
2018-10-23 14:30:11,201 : INFO : EPOCH 1 - PROGRESS: at 91.14% examples, 782698 words/s, in_qsize 18, out_qsize 1
2018-10-23 14:30:12,230 : INFO : EPOCH 1 - PROGRESS: at 94.00% examples, 783744 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:30:13,233 : INFO : EPOCH 1 - PROGRESS: at 97.07% examples, 786656 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:30:14,188 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-10-23 14:30:14,198 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-10-23 14:30:14,202 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-10-23 14:30:14,213 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-10-23 14:30:14,224 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-10-23 14:30:14,232 : INFO : worker thread finished; awaiting finish of 4 more thread

...

2018-10-23 14:32:00,964 : INFO : EPOCH 5 - PROGRESS: at 6.06% examples, 934217 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:32:01,966 : INFO : EPOCH 5 - PROGRESS: at 9.03% examples, 934165 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:32:02,977 : INFO : EPOCH 5 - PROGRESS: at 11.19% examples, 901204 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:32:03,981 : INFO : EPOCH 5 - PROGRESS: at 13.13% examples, 861435 words/s, in_qsize 18, out_qsize 1
2018-10-23 14:32:04,984 : INFO : EPOCH 5 - PROGRESS: at 15.73% examples, 863903 words/s, in_qsize 16, out_qsize 3
2018-10-23 14:32:05,996 : INFO : EPOCH 5 - PROGRESS: at 18.78% examples, 899283 words/s, in_qsize 18, out_qsize 1
2018-10-23 14:32:07,001 : INFO : EPOCH 5 - PROGRESS: at 21.65% examples, 912280 words/s, in_qsize 14, out_qsize 5
2018-10-23 14:32:08,021 : INFO : EPOCH 5 - PROGRESS: at 24.17% examples, 917503 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:32:09,040 : INFO : EPOCH 5 - PROGRESS: at 27.96% examples, 930056 words/s, i

2018-10-23 14:32:55,980 : INFO : EPOCH - 6 : training on 41519355 raw words (30346277 effective words) took 27.5s, 1104001 effective words/s
2018-10-23 14:32:56,995 : INFO : EPOCH 7 - PROGRESS: at 3.64% examples, 1113917 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:32:57,998 : INFO : EPOCH 7 - PROGRESS: at 7.35% examples, 1127957 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:32:59,005 : INFO : EPOCH 7 - PROGRESS: at 10.61% examples, 1125585 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:33:00,017 : INFO : EPOCH 7 - PROGRESS: at 13.75% examples, 1122790 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:33:01,022 : INFO : EPOCH 7 - PROGRESS: at 17.05% examples, 1129086 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:33:02,023 : INFO : EPOCH 7 - PROGRESS: at 20.04% examples, 1130806 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:33:03,030 : INFO : EPOCH 7 - PROGRESS: at 23.26% examples, 1129328 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:33:04,037 : INFO : EPOCH 7 - PROGRESS: at 

2018-10-23 14:34:00,281 : INFO : EPOCH 8 - PROGRESS: at 97.26% examples, 886573 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:34:01,205 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-10-23 14:34:01,233 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-10-23 14:34:01,246 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-10-23 14:34:01,259 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-10-23 14:34:01,271 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-10-23 14:34:01,273 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-10-23 14:34:01,277 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-10-23 14:34:01,283 : INFO : EPOCH 8 - PROGRESS: at 99.96% examples, 883272 words/s, in_qsize 2, out_qsize 1
2018-10-23 14:34:01,284 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-10-23 14:34:01,285 : INFO : worker threa

2018-10-23 14:34:55,591 : INFO : EPOCH 10 - PROGRESS: at 70.34% examples, 928718 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:34:56,602 : INFO : EPOCH 10 - PROGRESS: at 73.62% examples, 928698 words/s, in_qsize 18, out_qsize 1
2018-10-23 14:34:57,605 : INFO : EPOCH 10 - PROGRESS: at 76.51% examples, 928032 words/s, in_qsize 18, out_qsize 1
2018-10-23 14:34:58,632 : INFO : EPOCH 10 - PROGRESS: at 79.59% examples, 927912 words/s, in_qsize 17, out_qsize 2
2018-10-23 14:34:59,633 : INFO : EPOCH 10 - PROGRESS: at 82.28% examples, 924183 words/s, in_qsize 19, out_qsize 6
2018-10-23 14:35:00,641 : INFO : EPOCH 10 - PROGRESS: at 85.27% examples, 924036 words/s, in_qsize 18, out_qsize 1
2018-10-23 14:35:01,665 : INFO : EPOCH 10 - PROGRESS: at 88.48% examples, 922522 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:35:02,670 : INFO : EPOCH 10 - PROGRESS: at 91.79% examples, 922917 words/s, in_qsize 19, out_qsize 0
2018-10-23 14:35:03,682 : INFO : EPOCH 10 - PROGRESS: at 95.04% examples, 923758

(303488249, 415193550)

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [5]:

w1 = "dirty"
model.wv.most_similar (positive=w1)


2018-10-23 14:35:05,088 : INFO : precomputing L2-norms of word weight vectors
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('filthy', 0.8684808611869812),
 ('stained', 0.7797233462333679),
 ('unclean', 0.7728196978569031),
 ('smelly', 0.7563701868057251),
 ('dusty', 0.7521894574165344),
 ('grubby', 0.7508471012115479),
 ('grimy', 0.7466033697128296),
 ('dingy', 0.7357178330421448),
 ('soiled', 0.7203205227851868),
 ('gross', 0.7166814804077148)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [6]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('courteous', 0.9128109216690063),
 ('friendly', 0.8269704580307007),
 ('cordial', 0.7921279072761536),
 ('professional', 0.7823814749717712),
 ('attentive', 0.7719215154647827),
 ('curteous', 0.7616638541221619)]

In [7]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('canada', 0.696135401725769),
 ('germany', 0.6570076942443848),
 ('hawaii', 0.6482532024383545),
 ('england', 0.6377837657928467),
 ('spain', 0.6188236474990845),
 ('gaulle', 0.6184977889060974)]

In [8]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('horrified', 0.8035989999771118),
 ('amazed', 0.7973026037216187),
 ('astonished', 0.7695813179016113),
 ('appalled', 0.7583582401275635),
 ('dismayed', 0.7440380454063416),
 ('stunned', 0.7386012077331543)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [9]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('duvet', 0.7165772914886475),
 ('mattress', 0.7019136548042297),
 ('blanket', 0.7014796137809753),
 ('quilt', 0.6976365447044373),
 ('pillowcase', 0.6968289017677307),
 ('matress', 0.6824468374252319),
 ('pillows', 0.6468196511268616),
 ('sheets', 0.6290844678878784),
 ('foam', 0.6226242780685425),
 ('comforter', 0.6007372736930847)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [10]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.75637025

In [11]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.99999994

In [12]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.27014747

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [13]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'france'

In [14]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'shower'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?

### `sg`
Set sg to 1 to use the skip gram model and set it to 0 to use CBOW. By default, sg is set to 0

## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
